# Pré-processamento

Notebook: [notebooks/02_preprocessing.ipynb](../report/02_preprocessing.md)

In [1]:
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler, StandardScaler

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))

input_dir = os.path.join(project_root, "data", "processed")
minmax_dir = os.path.join(project_root, "data", "final", "minmax")
standard_dir = os.path.join(project_root, "data", "final", "standard")

os.makedirs(minmax_dir, exist_ok=True)
os.makedirs(standard_dir, exist_ok=True)

files = [
    "KDDTrain+.csv",
    "KDDTest+.csv",
    "KDDTrain+_20Percent.csv",
    "KDDTest-21.csv"
]


## Mapeamento das Classes de Ataque

Agrupamento dos rótulos originais em 5 categorias principais, conforme metodologia do artigo base.

In [2]:
attack_map = {
    'normal': 'normal',
    # DoS
    'back': 'DoS', 'land': 'DoS', 'neptune': 'DoS', 'pod': 'DoS', 'smurf': 'DoS', 'teardrop': 'DoS',
    'apache2': 'DoS', 'udpstorm': 'DoS', 'processtable': 'DoS', 'mailbomb': 'DoS',
    # Probe
    'satan': 'Probe', 'ipsweep': 'Probe', 'nmap': 'Probe', 'portsweep': 'Probe', 'mscan': 'Probe', 'saint': 'Probe',
    # R2L
    'ftp_write': 'R2L', 'guess_passwd': 'R2L', 'imap': 'R2L', 'multihop': 'R2L', 'phf': 'R2L', 'spy': 'R2L',
    'warezclient': 'R2L', 'warezmaster': 'R2L', 'xlock': 'R2L', 'xsnoop': 'R2L', 'snmpguess': 'R2L',
    'snmpgetattack': 'R2L', 'httptunnel': 'R2L', 'sendmail': 'R2L', 'named': 'R2L',
    # U2R
    'buffer_overflow': 'U2R', 'loadmodule': 'U2R', 'perl': 'U2R', 'rootkit': 'U2R',
    'ps': 'U2R', 'sqlattack': 'U2R', 'xterm': 'U2R'
}

## Função de Pré-processamento

A função abaixo executa:
- Remoção da coluna `difficulty` e `num_outbound_cmds`
- Agrupamento dos serviços menos frequentes em "other"
- One-hot encoding das variáveis categóricas
- Normalização MinMax e Standard das variáveis numéricas
- Salvamento dos arquivos finais prontos para modelagem

In [3]:
def preprocess_dual_scaling(file_name):
    df = pd.read_csv(os.path.join(input_dir, file_name))

    if "difficulty" in df.columns:
        df.drop(columns=["difficulty"], inplace=True)

    # Remover coluna com variância zero
    if "num_outbound_cmds" in df.columns:
        df.drop(columns=["num_outbound_cmds"], inplace=True)

    # Criar coluna de categoria
    df["attack_category"] = df["class"].map(attack_map)

    # Agrupar serviços menos comuns
    top_services = df["service"].value_counts().nlargest(10).index
    df["service"] = df["service"].apply(lambda x: x if x in top_services else "other")

    # One-hot nas categóricas
    df = pd.get_dummies(df, columns=["protocol_type", "flag", "service"], drop_first=False)

    # Separar features e target
    y = df["attack_category"]
    X = df.drop(columns=["class", "attack_category"])

    # Selecionar colunas numéricas
    numeric_cols = X.select_dtypes(include=["int64", "float64"]).columns

    # Normalizações
    for scaler, out_dir, label in [
        (MinMaxScaler(), minmax_dir, "minmax"),
        (StandardScaler(), standard_dir, "standard")
    ]:
        X_scaled = X.copy()
        X_scaled[numeric_cols] = scaler.fit_transform(X_scaled[numeric_cols])

        df_final = pd.concat([X_scaled, y], axis=1)

        # 🔁 Convertendo booleanos para inteiros (0/1)
        df_final = df_final.astype({col: int for col in df_final.select_dtypes('bool').columns})

        out_name = file_name.replace(".csv", f"_final_{label}.csv")
        out_path = os.path.join(out_dir, out_name)
        df_final.to_csv(out_path, index=False)
        print(f"Salvo: {out_path}")


## Execução

In [4]:
        
for f in files:
    preprocess_dual_scaling(f)

Salvo: /home/corisco/cesar-school/ml-ids/data/final/minmax/KDDTrain+_final_minmax.csv
Salvo: /home/corisco/cesar-school/ml-ids/data/final/standard/KDDTrain+_final_standard.csv
Salvo: /home/corisco/cesar-school/ml-ids/data/final/minmax/KDDTest+_final_minmax.csv
Salvo: /home/corisco/cesar-school/ml-ids/data/final/standard/KDDTest+_final_standard.csv
Salvo: /home/corisco/cesar-school/ml-ids/data/final/minmax/KDDTrain+_20Percent_final_minmax.csv
Salvo: /home/corisco/cesar-school/ml-ids/data/final/standard/KDDTrain+_20Percent_final_standard.csv
Salvo: /home/corisco/cesar-school/ml-ids/data/final/minmax/KDDTest-21_final_minmax.csv
Salvo: /home/corisco/cesar-school/ml-ids/data/final/standard/KDDTest-21_final_standard.csv
